In [1]:
import stella
import numpy as np
import matplotlib.pyplot as plt
from lightkurve.lightcurve import LightCurve as LC
from lightkurve.search import search_targetpixelfile

In [2]:
tic = 220433363

stpf = search_targetpixelfile('TIC'+str(tic), mission='TESS')
tpfs = stpf.download_all()

time, flux, fluxerr = [], [], []
for d in tpfs.data:
    lc = d.to_lightcurve()
    
    q = lc.quality == 0
    
    time.append(lc.time[q])
    flux.append(lc.flux[q])
    fluxerr.append(lc.flux_err[q])
    
time = np.array(time)
flux = np.array(flux)
fluxerr = np.array(fluxerr)

W1004 11:01:57.551271 4602336704 utils.py:96] Warning: 31% (6168/19692) of the cadences will be ignored due to the quality mask (quality_bitmask=175).


# Creating the simulated data set

In [11]:
slc = stella.SimulateLightCurves(sample_size=3000)
slc.sine_wave()
slc.inject_flares()

100%|██████████| 3000/3000 [06:04<00:00,  8.23it/s]


# Building the network

In [ ]:
model = stella.NeuralNetwork(slc)
model.network_model()
model.train_data(epochs=250)

Epoch 1/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.4642 - acc: 0.8032
Epoch 2/250
112147/112147 [==============================] - 2s 22us/sample - loss: 0.2487 - acc: 0.9175
Epoch 3/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.2290 - acc: 0.9259
Epoch 4/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.2200 - acc: 0.9301
Epoch 5/250
112147/112147 [==============================] - 2s 22us/sample - loss: 0.2452 - acc: 0.9236
Epoch 6/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.2485 - acc: 0.9209
Epoch 7/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.2223 - acc: 0.9292
Epoch 8/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.2532 - acc: 0.9164
Epoch 9/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.2632 - acc: 0.9133
Epoch 10/250
112147/112147 [===========================

112147/112147 [==============================] - 2s 20us/sample - loss: 0.1640 - acc: 0.9513
Epoch 79/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.1714 - acc: 0.9490
Epoch 80/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1809 - acc: 0.9444
Epoch 81/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1545 - acc: 0.9538
Epoch 82/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1597 - acc: 0.9519
Epoch 83/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1573 - acc: 0.9525
Epoch 84/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1621 - acc: 0.9517
Epoch 85/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1521 - acc: 0.9543
Epoch 86/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1728 - acc: 0.9480
Epoch 87/250
112147/112147 [==============================]

112147/112147 [==============================] - 2s 21us/sample - loss: 0.1376 - acc: 0.9592
Epoch 155/250
112147/112147 [==============================] - 2s 21us/sample - loss: 0.1375 - acc: 0.9594
Epoch 156/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1441 - acc: 0.9572
Epoch 157/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1394 - acc: 0.9584
Epoch 158/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1400 - acc: 0.9586
Epoch 159/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1392 - acc: 0.9589
Epoch 160/250
112147/112147 [==============================] - 2s 22us/sample - loss: 0.1397 - acc: 0.9585
Epoch 161/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1325 - acc: 0.9609
Epoch 162/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1423 - acc: 0.9571
Epoch 163/250
112147/112147 [======================

112147/112147 [==============================] - 2s 20us/sample - loss: 0.1341 - acc: 0.9601
Epoch 231/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1301 - acc: 0.9618
Epoch 232/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1373 - acc: 0.9595
Epoch 233/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1320 - acc: 0.9611
Epoch 234/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1313 - acc: 0.9606
Epoch 235/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1330 - acc: 0.9606
Epoch 236/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1303 - acc: 0.9615
Epoch 237/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1322 - acc: 0.9608
Epoch 238/250
112147/112147 [==============================] - 2s 20us/sample - loss: 0.1311 - acc: 0.9616
Epoch 239/250
112147/112147 [======================

# Predicting real data

In [ ]:
model.predict(time[0], flux[0], fluxerr[0])

# The Results

In [ ]:
plt.figure(figsize=(14,8))
plt.plot(model.time[0], model.flux[0]/np.nanmedian(model.flux[0]), 'k', label='flux')
plt.plot(model.time[0], model.detrended_flux[0]-0.2, 'darkorange', label='detrended')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,8))
plt.scatter(model.time[0], model.flux[0],
           c=model.predictions[0][:,1], vmin=0, vmax=1)
plt.colorbar(label='Probability');